- Gerardo Peña Pérez
- Carlos Alberto 
- Kevin Joan 
- Wenguang Hu

In [1]:
import torch
print(torch.__version__)

1.13.0+cu117


In [2]:
!pip uninstall torchtext 
!pip install torchtext

Found existing installation: torchtext 0.14.0
Uninstalling torchtext-0.14.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchtext-0.14.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchtext/*
Proceed (y/n)? y
  Successfully uninstalled torchtext-0.14.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torchtext-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (2.0 MB)


In [3]:
!pip install torch==1.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.5.0 requires torch==1.13.0, but you have torch 1.12.1 which is incompatible.


In [3]:
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
from torch import nn
from torch.nn import functional as F
# Dataset and dataloader
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

In [4]:
torch.cuda.is_available()

True

In [5]:
# Use Gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
train_dataset,  test_dataset = AG_NEWS()

In [7]:
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [8]:
len(train_dataset), len(test_dataset)

(120000, 7600)

In [9]:
train_dataset[:10]

[(3,
  "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."),
 (3,
  'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'),
 (3,
  "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums."),
 (3,
  'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.'),
 (3,
  'Oil prices soar to all-time record, 

In [10]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
  for _, text in data:
    yield tokeniser(text)

In [11]:
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [12]:
len(vocab)

95811

In [13]:
for i in range(20):
  print(i, vocab.lookup_token(i))

0 <unk>
1 .
2 the
3 ,
4 to
5 a
6 of
7 in
8 and
9 s
10 on
11 for
12 #39
13 (
14 )
15 -
16 '
17 that
18 with
19 as


In [14]:
tokens = tokeniser('Welcome to TE3007B')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007b'] [3314, 4, 0]


In [15]:
NUM_TRAIN = int(len(train_dataset) * 0.9)
NUM_VAL =len(train_dataset) - NUM_TRAIN

In [16]:
NUM_VAL

12000

In [17]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [18]:
len(train_dataset), len(val_dataset), len(test_dataset)

(108000, 12000, 7600)

In [19]:
max_tokens = 50

In [20]:
def collate_batch(batch):
  y, x = list(zip(*batch))
  #create list with tokens
  x = [vocab(tokeniser(text)) for text in x]
  # padding or clipping
  x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]
  return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)-1

In [21]:
labels = ['World', 'Sports', 'Business', 'Sci/Tech']
BATCH_SIZE = 1024

In [22]:
# DataLoaders
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)
val_loader = DataLoader(val_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)
test_loader = DataLoader(test_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)

In [23]:
for i, (x, y) in enumerate(test_loader):
  print(i, x.shape, y.shape)

0 torch.Size([1024, 50]) torch.Size([1024])
1 torch.Size([1024, 50]) torch.Size([1024])
2 torch.Size([1024, 50]) torch.Size([1024])
3 torch.Size([1024, 50]) torch.Size([1024])
4 torch.Size([1024, 50]) torch.Size([1024])
5 torch.Size([1024, 50]) torch.Size([1024])
6 torch.Size([1024, 50]) torch.Size([1024])
7 torch.Size([432, 50]) torch.Size([432])


In [24]:
#let us build our RNN

In [43]:
EMBEDDING_SIZE = 300
NEURONS = 400 #hidden
LAYERS = 4
NUM_CLASSES = 4

In [44]:
class RNN_Model_1(nn.Module):
  def __init__(self, embed_size, hidden, layers, num_classes):
    super().__init__()
    self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                        embedding_dim=embed_size)
    
    self.rnn = nn.GRU(input_size=embed_size,
                      hidden_size = hidden,
                      num_layers = layers,
                      batch_first = True)

    self.fc = nn.Linear(in_features=hidden, out_features= num_classes)

  def forward(self, x):
    vector_embs = self.embedding_layer(x)
    y, h = self.rnn(vector_embs)
    return self.fc(y[:,-1])
    

In [45]:
rnn_model = RNN_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)

In [46]:
rnn_model

RNN_Model_1(
  (embedding_layer): Embedding(95811, 300)
  (rnn): GRU(300, 400, num_layers=4, batch_first=True)
  (fc): Linear(in_features=400, out_features=4, bias=True)
)

In [47]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype = torch.long)
            y = y.to(device=device, dtype = torch.long)
            scores = model(x)
            _, preds = scores.max(dim=1)
            num_correct += (preds == y).sum()
            num_total += preds.size(0)
        acc = float(num_correct)/num_total
        return acc

In [48]:
def train(model, optimiser, epochs=100):
  model = model.to(device=device)
  for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
      model.train()
      x = x.to(device=device, dtype=torch.long)
      y = y.to(device=device, dtype=torch.long)
      # run model
      scores = model(x)
      # compute cost
      cost = F.cross_entropy(input=scores, target=y)
      # reset gradient
      optimiser.zero_grad()
      # compute gradient
      cost.backward()
      # update parameter
      optimiser.step()
    acc = accuracy(model, val_loader)
    print(f'Epoch {epoch}, costo {cost.item():.4f}, val acc {acc:.4f}')


In [49]:
epochs = 10
lr = 0.0004

rnn_model = RNN_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)
optimiser = torch.optim.Adam(rnn_model.parameters(), lr=lr)

In [50]:
train(rnn_model, optimiser, epochs)

Epoch 0, costo 0.4922, val acc 0.8217
Epoch 1, costo 0.3511, val acc 0.8822
Epoch 2, costo 0.2765, val acc 0.8912
Epoch 3, costo 0.2301, val acc 0.8912
Epoch 4, costo 0.1957, val acc 0.8975
Epoch 5, costo 0.1651, val acc 0.9017
Epoch 6, costo 0.1333, val acc 0.9023
Epoch 7, costo 0.1237, val acc 0.9076
Epoch 8, costo 0.1103, val acc 0.9033
Epoch 9, costo 0.0644, val acc 0.9044


In [51]:
print(f'{accuracy(rnn_model, test_loader):.4f}')

0.9041
